In [30]:
import requests
import json
import pandas as pd
import time
from tqdm import tqdm_notebook
from selenium import webdriver

In [11]:
naver_data = pd.read_csv('./data/meamul_up_to_date_detail.csv')
landbook = pd.read_csv('./data/landbook.csv')

In [132]:
temp = naver_data[['latitude','longitude','pnu']]

In [133]:
temp.drop_duplicates(subset=['latitude','longitude'], inplace= True)
temp = temp.reset_index(drop = True)

C:\Users\gongdan\anaconda3\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [26]:
temp = []
for i in tqdm_notebook(range(0, len(naver_data))):
    temp.append(str(naver_data['latitude'][i])+', '+str(naver_data['longitude'][i]))


C:\Users\gongdan\AppData\Local\Temp/ipykernel_12976/1694063724.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(0, len(naver_data))):


  0%|          | 0/1438 [00:00<?, ?it/s]

In [32]:
driver = webdriver.Chrome('./chromedriver_win.exe')
driver.get('https://www.google.com/maps/?hl=ko')
driver.find_element_by_xpath('/html/body/div[3]/div[9]/div[3]/div[1]/div[1]/div[1]/div[2]/form/div/div[3]/div/input[1]').send_keys(i)
temp = pd.DataFrame(temp).drop_duplicates()

In [50]:
google_juso = []
for i in tqdm_notebook(temp[0]):
    driver.find_element_by_xpath('/html/body/div[3]/div[9]/div[3]/div[1]/div[1]/div[1]/div[2]/form/div/div[3]/div/input[1]').clear()
    driver.find_element_by_xpath('/html/body/div[3]/div[9]/div[3]/div[1]/div[1]/div[1]/div[2]/form/div/div[3]/div/input[1]').send_keys(i)
    driver.find_element_by_xpath('/html/body/div[3]/div[9]/div[3]/div[1]/div[1]/div[1]/div[2]/div[1]/button').click()
    time.sleep(3)
    google_juso.append(driver.find_element_by_xpath('/html/body/div[3]/div[9]/div[8]/div/div[1]/div/div/div[7]/div/div[1]/span[3]/span[3]').text)
    

In [143]:
df = pd.merge(temp, landbook, left_on=['latitude','longitude'], right_on=['naver_latitude','naver_longitude'], how = 'outer')

In [147]:
find_landbook = df[df['id'].isna() == True]
find_landbook = find_landbook[['latitude_x','longitude_x','pnu']]
find_landbook.columns = ['naver_latitude','naver_longitude','naver_pnu']

In [156]:
result_landbook = []
for idx, key in enumerate(tqdm_notebook(find_landbook.index)):
    url = 'https://api2.landbook.net/point_polygons?lon={lon}&lat={lat}'.format(lon = find_landbook['naver_longitude'][key], lat = find_landbook['naver_latitude'][key])
    r = requests.get(url, timeout = 5)
    result_landbook. append([json.loads(r.text)['id'], find_landbook['naver_longitude'][key], find_landbook['naver_latitude'][key],find_landbook['naver_pnu'][key]])
    time.sleep(2)

C:\Users\gongdan\AppData\Local\Temp/ipykernel_12976/940860847.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for idx, key in enumerate(tqdm_notebook(find_landbook.index)):


  0%|          | 0/217 [00:00<?, ?it/s]

In [162]:
result_landbook = pd.DataFrame(result_landbook)
result_landbook.columns = ['landbook_pnu','naver_longitude','naver_latitude','naver_pnu']

In [163]:
result_landbook

,landbook_pnu,naver_longitude,naver_latitude,naver_pnu
0,2823710500104600003,126.735422,37.507883,2.823711e+18
1,2823710500101220013,126.744641,37.518396,2.823711e+18
2,2823710500104600004,126.735739,37.507867,2.823711e+18
3,2823710500104600001,126.734781,37.507914,2.823711e+18
4,2823710500104640003,126.733541,37.507264,2.823711e+18
...,...,...,...,...
212,2824510400102080001,126.752062,37.537082,2.824510e+18
213,2824510400102090000,126.756501,37.537191,2.824510e+18
214,2824510300109060006,126.736062,37.533263,2.824510e+18
215,2824510300107380003,126.731423,37.524465,2.824510e+18


In [ ]:
## 기존 landbook 데이터와 신규로 찾을 데이터 비교 과정 필요
# need_search_jibeon = 

In [481]:
result_jibeon = pd.DataFrame()
for idx, item in tqdm.tqdm_notebook(enumerate(jibeon['pnu'])):
    
    url = 'https://api2.landbook.net/parcels/{num}'.format(num = item)
    r = requests.get(url, timeout = 5)
    try:
        info = json.loads(r.text)['parcel_info']
        info.pop('geometry')
        info.pop('building_list')
        info.pop('other_parcel')
        info = dict(info)
        temp = pd.DataFrame(info,  index = [0])
        result_jibeon = pd.concat([result_jibeon, temp])
        time.sleep(2)
    except:
        print(idx, "error", item)
result_jibeon = result_jibeon.reset_index(drop = True)

C:\Users\gongdan\AppData\Local\Temp/ipykernel_9708/2791178482.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for idx, item in tqdm.tqdm_notebook(enumerate(jibeon['pnu'])):


0it [00:00, ?it/s]

In [482]:
result_jibeon = result_jibeon.reset_index(drop = True)

In [483]:
result_jibeon[['naver_longitude','naver_latitude','naver_pnu']] = jibeon[['longitude','latitude','naver_pnu']]

In [484]:
landbook = pd.read_csv('./data/landbook.csv')

In [485]:
result = pd.concat([landbook, result_jibeon])

In [486]:
result.to_csv('./data/landbook.csv', index = False, encoding = "utf-8-sig")